In [1]:
using JuMP
using Cbc
using GLPK
using ECOS
using HiGHS
using MAT
using SparseArrays
using DelimitedFiles
using LinearAlgebra
using MathOptInterface

In [2]:
# get inputs
vars = matread("R3T3.mat")
S = vars["S"]
U = vars["U"]
L = vars["L"]

13094×50 SparseMatrixCSC{Float64, Int64} with 42839 stored entries:
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿

In [9]:
# first use l1 as a estimation of l0 

size_of_v_r = 13094
size_of_v_c = 50

# define the problem
model = JuMP.Model(Cbc.Optimizer)

@variable(model, v[1:size_of_v_r, 1:size_of_v_c])  # define variable v
@variable(model, v_norm[1:size_of_v_r])
println("variables declared")
for i in 1:size_of_v_r
    for j in 1:size_of_v_c
        @constraint(model, v_norm[i] >= v[i,j])
        @constraint(model, v_norm[i] >= -v[i,j])
    end
end
println("v_norm constraints declared")
@constraint(model, Matrix(L) .<= v .<= Matrix(U))  # l1 <= v <= u1
@constraint(model, Matrix(S) * v .== 0)  # Sv = 0
println("main constraints declared")
@objective(model, Min, sum(v_norm[i] for i in 1:size_of_v_r))  # min ||v||_1
print("model created")



variables declared
v_norm constraints declared
main constraints declared
model created

In [10]:
model

A JuMP Model
Minimization problem with:
Variables: 667794
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 420200 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 1309400 constraints
`AffExpr`-in-`MathOptInterface.Interval{Float64}`: 654700 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)
Names registered in the model: v, v_norm

In [11]:
# solve the problem, see the status of it, and store the results
optimize!(model)
println(termination_status(model))
print(primal_status(model))
starting_v_norm = JuMP.value.(v_norm)
starting_v = JuMP.value.(v)
sparse(starting_v)

Presolve 9210 (-2375090) rows, 7228 (-660566) columns and 40484 (-6029666) elements
0  Obj 51.942401 Primal inf 3416.6874 (5863) Dual inf 5.5857e+13 (2641)
317  Obj 51.942401 Primal inf 3179.0566 (5628) Dual inf 5.6024683e+13 (2470)
End of values pass after 445 iterations
445  Obj 51.942401 Primal inf 3098.1598 (5540) Dual inf 5.5874233e+13 (2397) w.o. free dual inf (2395)
End of values pass after 445 iterations
445  Obj 51.942401 Primal inf 3098.1598 (5540) Dual inf 5.5874233e+13 (2397) w.o. free dual inf (2394)
End of values pass after 445 iterations
445  Obj 51.942401 Primal inf 3098.1598 (5540) Dual inf 5.5874233e+13 (2397) w.o. free dual inf (2393)
End of values pass after 445 iterations
445  Obj 51.942401 Primal inf 3098.1598 (5540) Dual inf 5.5874233e+13 (2397) w.o. free dual inf (2392)
End of values pass after 445 iterations
445  Obj 51.942401 Primal inf 3098.1598 (5540) Dual inf 5.5874233e+13 (2397) w.o. free dual inf (2391)
End of values pass after 445 iterations
445  Obj 51.

Perturbing problem by 0.001% of 0.59241498 - largest nonzero change 0 ( 0%) - largesOPTIMAL
FEASIBLE_POINTt zero change 2.9986437e-05
716  Obj 142.49186 Primal inf 1108.6486 (2761) Dual inf 6.3296667e+13 (2551) w.o. free dual inf (2501)
983  Obj 148.00893 Primal inf 849.06511 (2414) Dual inf 17548332 (2537) w.o. free dual inf (2493)
1250  Obj 157.91117 Primal inf 639.28496 (2055) Dual inf 15714452 (2531) w.o. free dual inf (2493)
1516  Obj 183.78384 Primal inf 462.63718 (1682) Dual inf 11922246 (2399) w.o. free dual inf (2364)
1780  Obj 195.86468 Primal inf 359.29105 (1330) Dual inf 11282757 (2246) w.o. free dual inf (2215)
2042  Obj 257.86948 Primal inf 230.65011 (948) Dual inf 8334478.1 (2232) w.o. free dual inf (2203)
2315  Obj 272.69347 Primal inf 160.38813 (720) Dual inf 7258249.3 (2150) w.o. free dual inf (2123)
2577  Obj 320.69045 Primal inf 94.700579 (479) Dual inf 6228102.3 (1945) w.o. free dual inf (1919)
2840  Obj 373.53372 Primal inf 34.440001 (248) Dual inf 5481803.6 (1825

13094×50 SparseMatrixCSC{Float64, Int64} with 5473 stored entries:
⣦
⠿
⣪
⠋
⣿
⣐
⣐
⣛
⡿
⠛
⣾
⣿
⣿
⢿
⠂
⠀
⣿
⠙
⣚
⣿
⠲
⠀
⣿
⠬
⡿
⣉

In [13]:
# save the naive output
DelimitedFiles.writedlm("starting_v.txt", starting_v, ',')
DelimitedFiles.writedlm("starting_v_norm.txt", starting_v_norm, ',')

In [9]:
# solve with iterations to get a better answer compared to the naive l1 norm approach
delta = 0.0001
n = 1
num_iteration = 3
print(sparse(starting_v))
#prepare the model (in the for loop we only change the objective of the model)
model = JuMP.Model(Cbc.Optimizer)

@variable(model, v[1:size_of_v_r,1:size_of_v_c])  # define variable v
@variable(model, v_norm[1:size_of_v_r])  # define variable v_norm
@constraint(model, Matrix(L) .<= v .<= Matrix(U))  # l1 <= v <= u1
@constraint(model, Matrix(S) * v .== 0)  # Sv = 0

for i in 1:size_of_v_r
    for j in 1:size_of_v_c
        @constraint(model, v_norm[i] >= v[i,j])
        @constraint(model, v_norm[i] >= -v[i,j])
    end
end

#iterate
for iteration in 1:num_iteration
    print("doing iteration ")
    println(iteration)
    
    # decrement n
    n -= 0.5 / num_iteration
    
    w = zeros(size_of_v_r)
    for i in 1:size_of_v_r
        w[i] = delta ^ n / (delta ^ n + abs(starting_v_norm[i])^n)
    end
    
    # define the problem
    for i in 1:size_of_v_r
        for j in 1:size_of_v_c
            JuMP.set_start_value(v[i, j], starting_v[i, j])
        end
    end
    for i in 1:size_of_v_r
        JuMP.set_start_value(v_norm[i], starting_v_norm[i])
    end
    for i in 1:size_of_v_r
        if abs(starting_v_norm[i]) < 0.00002
            @constraint(model, v_norm[i] == starting_v_norm[i])
        end
    end
    @objective(model, Min, sum(v_norm[i] * w[i] for i in 1:size_of_v_r))  # min ||v||_1
    
    # solve the problem, check the status of it, and update starting_v and starting_v_norm
    optimize!(model)
    println(termination_status(model))
    println(primal_status(model))
    starting_v = JuMP.value.(v)
    starting_v_norm = JuMP.value.(v_norm)
    println("before saving")
    DelimitedFiles.writedlm(string(path, "output_iteration_", string(iteration), ".txt"), starting_v, ',')
    println("after saving")
end
sparse(starting_v)


⣦
⣛
⣍
⠦
⣿
⡉
⠄
⠀
⠉
⡭
⣽
⣺
⣿
⣴
⠟
⣀
⠀
⠀
⣀
⣬
⠛
⣾
⣿
⣿
⣿
⣿doing iteration 1
OPTIMAL
FEASIBLE_POINT
doing iteration 2
Presolve 4237 (-735442) rows, 2873 (-203680) columns and 17309 (-1817910) elements
Perturbing problem by 0.001% of 0.11181907 - largest nonzero change 0.00012545886 ( 43.254573%) - largest zero change 0.00012541338
0  Obj 0.16842677 Primal inf 882.2492 (2166)
159  Obj 0.17241145 Primal inf 756.01815 (2108)
318  Obj 0.17546503 Primal inf 674.97695 (2038)
477  Obj 0.18023367 Primal inf 570.76432 (1916)
636  Obj 0.18554139 Primal inf 561.72579 (1840)
795  Obj 0.18992477 Primal inf 409.30397 (1638)
954  Obj 0.19546722 Primal inf 308.41809 (1428)
1113  Obj 0.20091532 Primal inf 300.89234 (1284)
1272  Obj 0.20555672 Primal inf 220.59908 (1093)
1431  Obj 0.20920315 Primal inf 252.65073 (963)
1590  Obj 0.21576782 Primal inf 101.13983 (616)
1749  Obj 0.22084088 Primal inf 50.256922 (429)
1908  Obj 0.2272901 Primal inf 18.131517 (226)
2067  Obj 0.22917451 Primal inf 1.6048754 (73)
2141 

6663×30 SparseMatrixCSC{Float64, Int64} with 3120 stored entries:
⣦
⣛
⣍
⠦
⣿
⡉
⠄
⠀
⠉
⡭
⣽
⣺
⣿
⣴
⠟
⣀
⠀
⠀
⣀
⣬
⠛
⣾
⣿
⣿
⣿
⣿

In [10]:
# save the stronger output
DelimitedFiles.writedlm("output.txt", starting_v, ',')

In [4]:
# compute the bounds of l1 norm of x_i (MN stands for maximum norm)
MN = zeros(size_of_v_r)
for i in 1:size_of_v_r
    for j in 1:size_of_v_c
        MN[i] = max(abs(L[i, j]), abs(U[i, j]), MN[i])
    end
end
MN

13094-element Vector{Float64}:
 2.1690399722928957
 0.7979818884299401
 0.6034452341648391
 1.049327387497892
 1.722843680651103
 1.0140993341899616
 1.8921080897429328
 1.7834989549225733
 2.3186282587190174e-14
 2.2318574437653336
 0.887552481271632
 1.6339781474797015
 1.0021247830845401
 ⋮
 1.033591828000263
 8.000698154712564e-17
 2.0879247458562884
 0.8634368719891129
 1.8542278194565551
 1.7014803208455713
 1.0966976192345625
 0.7498155575470894
 1.2284105736717526e-15
 0.0
 2.303184195420016
 1.5479699019445032

In [5]:
# now solve the exact problem using MILP and with the starting points from previous part

# define the problem (we use Cbc because GLPK wrapper in JuMP doesnt support starting points)
model = JuMP.Model(Cbc.Optimizer)
@variable(model, v[1:size_of_v_r, 1:size_of_v_c])  # define variable v
for i in 1:size_of_v_r
    for j in 1:size_of_v_c
        JuMP.set_start_value(v[i, j], starting_v[i, j])
    end
end
@variable(model, v_norm[1:size_of_v_r])
for i in 1:size_of_v_r
    JuMP.set_start_value(v_norm[i], starting_v_norm[i])
end
@variable(model, b[1:size_of_v_r])
println("variables built")
@constraint(model, Matrix(L) .<= v .<= Matrix(U))  # l <= v <= u
@constraint(model, Matrix(S) * v .== 0)  # Sv = 0

@objective(model, Min, sum(b[i] for i in 1:size_of_v_r))  # min ||v||_0

for i in 1:size_of_v_r
    for j in 1:size_of_v_c
        @constraint(model, v_norm[i] >= v[i, j])
        @constraint(model, v_norm[i] >= -v[i, j])
    end
end
for i in 1:size_of_v_r
    @constraint(model, -MN[i] * b[i] <= v_norm[i])
    @constraint(model, v_norm[i] <= MN[i]*b[i])
    @constraint(model, b[i] in MathOptInterface.ZeroOne())
end
println("model created")

# solve the problem, print its status, and store the results
optimize!(model)
println(termination_status(model))
println(primal_status(model))
starting_v = JuMP.value.(v)

what is happening :/
haha0!
variable 1
variables built
haha3
haha1
haha2
model created


In [1]:
# save the final (exact) answer
DelimitedFiles.writedlm("output.txt", starting_v, ',')

LoadError: UndefVarError: DelimitedFiles not defined